# Training Dialogue Encoder

## Data Validation

## MultiWOZ Service Classification

In [1]:
from train_utils import MultiWOZServiceClfDataset

dataset = MultiWOZServiceClfDataset('/home/alekseev_ilya/dialogue-augmentation/dialogue_encoder/multiwoz_truncated/train')

/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset[0]

([{'utterance': 'i need a place to dine in the center thats expensive',
   'speaker': 0},
  {'utterance': 'I have several options for you; do you prefer African, Asian, or British food?',
   'speaker': 1},
  {'utterance': 'Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation?',
   'speaker': 0},
  {'utterance': 'There is an Afrian place named Bedouin in the centre. How does that sound?',
   'speaker': 1},
  {'utterance': 'Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel?',
   'speaker': 0},
  {'utterance': "Bedouin's phone is 01223367660. As far as hotels go, I recommend the University Arms Hotel in the center of town.",
   'speaker': 1},
  {'utterance': 'Yes. Can you book it for me?', 'speaker': 0},
  {'utterance': 'Sure, when would you like that reservation?', 'speaker': 1},
  {'utterance': 'i want to book it for 2 people and 2 nights starting from saturday.',
   'speaker': 0}

In [8]:
from datasets import load_dataset

dataset = load_dataset('multi_woz_v22')['validation']
dataset[0]

{'dialogue_id': 'PMUL0698.json',
 'services': ['restaurant', 'train'],
 'turns': {'turn_id': ['0',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11'],
  'speaker': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
  'utterance': ["I'm looking for a local place to dine in the centre that serves chinese food.",
   'I have restaurants matching your criteria in all price ranges. Do you have a preference on price?',
   'I need the address, postcode and the price range.',
   'Ok how about Charlie Chan, located at Regent Street City Centre. Postcode is cb21db with a cheap price. Can I help you further today?',
   'I also need a train. The train should leave after 16:15 and should leave on sunday.',
   "Can I have more information for the train you're needing? Where are you departing from and arriving to?",
   'I am leaving from Cambridge and going to Norwich.',
   'I have train TR1840 leaving at 16:36 is that okay?',
   'book for 5 people and get me the reference nu

In [6]:
from train_utils import MultiWOZServiceClfDataset
from torch.utils.data import DataLoader

dataset = MultiWOZServiceClfDataset('validation')
loader = DataLoader(
    dataset,
    batch_size=2,
    num_workers=1,
    collate_fn=lambda x: x
)

In [7]:
next(iter(loader))

[([{'utterance': "I'm looking for a local place to dine in the centre that serves chinese food.",
    'speaker': 0},
   {'utterance': 'I have restaurants matching your criteria in all price ranges. Do you have a preference on price?',
    'speaker': 1},
   {'utterance': 'I need the address, postcode and the price range.',
    'speaker': 0},
   {'utterance': 'Ok how about Charlie Chan, located at Regent Street City Centre. Postcode is cb21db with a cheap price. Can I help you further today?',
    'speaker': 1},
   {'utterance': 'I also need a train. The train should leave after 16:15 and should leave on sunday.',
    'speaker': 0},
   {'utterance': "Can I have more information for the train you're needing? Where are you departing from and arriving to?",
    'speaker': 1},
   {'utterance': 'I am leaving from Cambridge and going to Norwich.',
    'speaker': 0},
   {'utterance': 'I have train TR1840 leaving at 16:36 is that okay?',
    'speaker': 1},
   {'utterance': 'book for 5 people and

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
hidden_size = 768
n_classes = 7

train_size = 8000
val_size = 1000

In [3]:
import torch
from torch import nn
import torch.nn.functional as F


@torch.no_grad()
def generate_dataset(train_size, val_size, hidden_size, n_classes):
    embeddings = torch.empty(train_size+val_size, hidden_size)
    embeddings.normal_()

    linear_mapper = nn.Linear(hidden_size, n_classes)
    logits = linear_mapper(embeddings)
    targets = logits.ge(0).to(torch.float)

    train_dataset = list(zip(embeddings[:train_size], targets[:train_size]))
    val_dataset = list(zip(embeddings[train_size:], targets[train_size:]))

    return train_dataset, val_dataset

train_dataset, val_dataset = generate_dataset(train_size, val_size, hidden_size, n_classes)

In [4]:
from train_utils import get_multiwoz_service_clf_score

score = get_multiwoz_service_clf_score(train_dataset, val_dataset)

/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name | Type   | Params
--------------------------------
0 | clf  | Linear | 5.4 K 
--------------------------------
5.4 K     Trainable params
0         Non-trainable params
5.4 K     Total params
0.022     Total estimated mo

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          result           │    0.8644020557403564     │
└───────────────────────────┴───────────────────────────┘

In [6]:
score

[{'result': 0.6062570810317993}]

## Model

In [ ]:
import os
import json


n_chunks = 80
chunks_original = sorted([filename for filename in os.listdir(path_original) if filename.endswith('.json')])[:n_chunks]
chunks_positive = sorted([filename for filename in os.listdir(path_positive) if filename.endswith('.json')])[:n_chunks]

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
torch.set_float32_matmul_precision('medium')    

In [ ]:
from nup.models.dialogue import SimpleDialogueEncoder


encoder_name = 'xlnet-base-cased'
model = SimpleDialogueEncoder(encoder_name)

In [ ]:
# from nup.models.dialogue import HSSAConfig, HSSADM
# config = HSSAConfig(
#     max_ut_embeddings=28,
#     casual_utterance_attention=False,
# )
# dialogue_model = HSSADM(
#     'microsoft/mpnet-base',
#     config,
#     pool_utterance_level=False,
#     pool_dialogue_level=True
# ).cuda()
# dialogue_model.requires_grad_(False)